# Getting Access

In [1]:
import requests
import pandas as pd
import openpyxl
from datetime import datetime
import time
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

#Password Reader
with open('pw.txt', 'r') as f:
    pw = f.read()

with open('username.txt', 'r') as f:
    username = f.read()

In [2]:
CLIENT_ID = '<ur client id here>'
SECRET_KEY = '<ur secret key here>'

# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': username,
        'password': pw}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
#headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}
headers['Authorization'] = f'bearer {TOKEN}'

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)
#Response 200 = all ok

<Response [200]>

In [3]:
#Check Access
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers).json()

{'is_employee': False,
 'seen_layout_switch': False,
 'has_visited_new_profile': False,
 'pref_no_profanity': True,
 'has_external_account': False,
 'pref_geopopular': '',
 'seen_redesign_modal': False,
 'pref_show_trending': True,
 'subreddit': {'default_set': True,
  'user_is_contributor': False,
  'banner_img': '',
  'restrict_posting': True,
  'user_is_banned': False,
  'free_form_reports': True,
  'community_icon': None,
  'show_media': True,
  'icon_color': '#00F4CB',
  'user_is_muted': None,
  'display_name': 'u_AffectionateBed24',
  'header_img': None,
  'title': '',
  'coins': 0,
  'previous_names': [],
  'over_18': False,
  'icon_size': [256, 256],
  'primary_color': '',
  'icon_img': 'https://www.redditstatic.com/avatars/defaults/v2/avatar_default_4.png',
  'description': '',
  'allowed_media_in_comments': [],
  'submit_link_label': '',
  'header_size': None,
  'restrict_commenting': False,
  'subscribers': 0,
  'submit_text_label': '',
  'is_default_icon': True,
  'link_fla

In [4]:
# we use this function to convert responses to dataframes
def df_from_response(res):
    # initialize temp dataframe for batch of data in response
    df = pd.DataFrame()

    # loop through each post pulled from res and append to df
    for post in res.json()['data']['children']:
        df = df.append({
            'subreddit': post['data']['subreddit']
            , 'title': post['data']['title']
            , 'selftext': post['data']['selftext']
            , 'upvote_ratio': post['data']['upvote_ratio']
            , 'author': (post['data']['author'])
            ,' author_flair_text': (post['data']['author_flair_text'])
            , 'ups': post['data']['ups']
            , 'downs': post['data']['downs']
            , 'score': post['data']['score']
            , 'kind': post['kind']
            , 'id': post['data']['id']
            , 'link_flair_css_class': post['data']['link_flair_css_class']
            , 'created_utc': datetime.fromtimestamp(post['data']['created_utc']).strftime('%Y-%m-%dT%H:%M:%SZ')
        }, ignore_index=True)
        
    return df

# Making Requests

In [15]:
# Getting by newest, last 1k post
data = pd.DataFrame()
params = {'limit': 100}
counter = 0

# loop through n times (each request gets 100 posts) 1800 times get 180K post
# Sleep a second bcoz reddit api limit is 60 requests a minute 
for i in range(180):
    # make request
    res = requests.get("https://oauth.reddit.com/r/stopdrinking/new",
                       headers=headers,
                       params=params)

    # get dataframe from response
    new_df = df_from_response(res)
    # take the final row (oldest entry)
    try:
        if counter >= 999:
            row = new_df.iloc[len(new_df)-7]
        elif counter >= 99:
            row = new_df.iloc[len(new_df)-5]
        elif counter >= 9:
            row = new_df.iloc[len(new_df)-3]
        else:
            row = new_df.iloc[len(new_df)-1]
    except IndexError:
        print("Erorr index")
        break

    # create fullname
    fullname = row['kind'] + '_' + row['id']
    # add/update fullname in params
    params['after'] = fullname
    
    # append new_df to data
    data = data.append(new_df, ignore_index=True)
    counter += 1
    print('Req no. {}'.format(counter) )
    time.sleep(1)

print(counter)
data.to_excel('raw_new_posts_from_SD.xlsx')

C:\Users\User\AppData\Local\Temp\ipykernel_12640\3845986696.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\User\AppData\Local\Temp\ipykernel_12640\3845986696.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\User\AppData\Local\Temp\ipykernel_12640\3845986696.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\User\AppData\Local\Temp\ipykernel_12640\3845986696.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\User\AppData\Local\Temp\ipykernel_12640\3845986696.py:8: FutureWarning: The frame.append method is deprecated and will 

Req no. 1


C:\Users\User\AppData\Local\Temp\ipykernel_12640\3845986696.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\User\AppData\Local\Temp\ipykernel_12640\3845986696.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\User\AppData\Local\Temp\ipykernel_12640\3845986696.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\User\AppData\Local\Temp\ipykernel_12640\3845986696.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\User\AppData\Local\Temp\ipykernel_12640\3845986696.py:8: FutureWarning: The frame.append method is deprecated and will 

Req no. 2


C:\Users\User\AppData\Local\Temp\ipykernel_12640\3845986696.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\User\AppData\Local\Temp\ipykernel_12640\3845986696.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\User\AppData\Local\Temp\ipykernel_12640\3845986696.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\User\AppData\Local\Temp\ipykernel_12640\3845986696.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\User\AppData\Local\Temp\ipykernel_12640\3845986696.py:8: FutureWarning: The frame.append method is deprecated and will 

...

Req no. 9


C:\Users\User\AppData\Local\Temp\ipykernel_12640\3845986696.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\User\AppData\Local\Temp\ipykernel_12640\3845986696.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\User\AppData\Local\Temp\ipykernel_12640\3845986696.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\User\AppData\Local\Temp\ipykernel_12640\3845986696.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\User\AppData\Local\Temp\ipykernel_12640\3845986696.py:8: FutureWarning: The frame.append method is deprecated and will 

Req no. 10


C:\Users\User\AppData\Local\Temp\ipykernel_12640\3845986696.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\User\AppData\Local\Temp\ipykernel_12640\3845986696.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\User\AppData\Local\Temp\ipykernel_12640\3628891560.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(new_df, ignore_index=True)


Req no. 11
Erorr index
11
